<a href="https://colab.research.google.com/github/NikithKumar02/Coursera-Web-Developer-Hands-On/blob/main/TextClassificationUsingRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import the packages

In [11]:
import os
import sys
import numpy as np

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Input,Flatten
from keras.layers import Conv1D,MaxPooling1D,Embedding
from keras.models import Model
from keras.utils.np_utils import to_categorical

In [13]:
MAX_SEQUENCE_LENGTH=1000
MAX_NB_WORDS=10000
EMBEDDING_DIM=100
VALIDATION_SPLIT=0.2


# Create the index of word and vector

In [14]:
embeddings_index={}
f=open('/content/glove.6B.100d.txt','r',encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeddings_index[word]=coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


# selected category


In [15]:
selected_categories = [
    'comp.graphics',
    'rec.motorcycles',
    'rec.sport.baseball',
    'misc.forsale',
    'sci.electronics',
    'sci.med',
    'talk.politics.guns',
    'talk.religion.misc']

# Download Data

In [16]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [17]:
print(list(newsgroups_train))
print(list(newsgroups_train.target_names))

['data', 'filenames', 'target_names', 'target', 'DESCR']
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [18]:
newsgroups_train = fetch_20newsgroups(subset = 'train', categories = selected_categories, remove = ('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset = 'test', categories = selected_categories, remove=('headers', 'footers', 'quotes'))

In [20]:
texts = newsgroups_train['data']
labels = newsgroups_train['target']

In [21]:
print(len(texts))
print(np.unique(labels))
print(labels)

4472
[0 1 2 3 4 5 6 7]
[6 3 0 ... 4 3 0]


In [22]:
texts = [t for t in texts]
print(type(texts[0]),texts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Text vectorization

In [23]:
tokenizer=Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)

In [24]:
word_index=tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 43747 unique tokens.


In [25]:
data = pad_sequences(sequences, maxlen=MAX_NB_WORDS)
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
print('Data validation split.')

Shape of data tensor: (4472, 10000)
Shape of label tensor: (4472, 8)
Data validation split.


# Divide data into training set and validation set

In [26]:
indices=np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT*data.shape[0])
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

# Create word embeddings

In [27]:
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # If the word is not in the word index, set all to 0.
        embedding_matrix[i] = embedding_vector
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_NB_WORDS,
                            trainable=False)
print("Initializing model...")

Initializing model...


# Initialize convolutional and pooling layers


In [31]:
sequence_input = Input(shape = (MAX_NB_WORDS,) ,dtype = 'int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128,5,activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128,5,activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(8, activation='softmax')(x)

In [29]:
model = Model(sequence_input,preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
print("Training Model...")

Training Model...


#Train Model

In [50]:
model.fit(x_train, y_train, batch_size = 128, epochs = 2, validation_data = (x_val, y_val))

Epoch 1/2
28/28 [==============================] - 305s 11s/step - loss: 1.9866 - acc: 0.1786 - val_loss: 1.9881 - val_acc: 0.1902
Epoch 2/2
28/28 [==============================] - 304s 11s/step - loss: 1.9482 - acc: 0.1864 - val_loss: 1.9978 - val_acc: 0.1924


# Score & Accuracy

In [51]:
predicted = model.predict(x_val)
score, acc = model.evaluate(x_val, y_val, batch_size=128)
print('Test score:', score)
print('Test accuracy:', acc)

7/7 [==============================] - 25s 4s/step - loss: 1.9978 - acc: 0.1924
Test score: 1.9978173971176147
Test accuracy: 0.1923937350511551
